In [ ]:
%load_ext autoreload
%autoreload 2
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## setup pydoxtools code access, data & gdrive

finally...  run pydoxtools

In [1]:
# make sure we set the environment variable before loading pydoxtools for the first time
# %env TRAINING_DATA_DIR=/content/gdrive/MyDrive/pydoxtools/training_data
from pydoxtools import random_data_generators, training

## prepare training

In [2]:
bg = random_data_generators.TextBlockGenerator.std_generator()
bg.classmap, bg.classmap_inv, bg.num_generators, bg.class_gen, bg.gen_mapping, bg.weights

({0: 'address', 1: 'unknown'},
 {'address': 0, 'unknown': 1},
 4,
 (<pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f5c21a84880>,
  <pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f5c21a84910>),
 {<pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f5c21a84880>: 'address',
  <pydoxtools.random_data_generators.RandomTextBlockGenerator at 0x7f5c21a84190>: 'unknown',
  <pydoxtools.random_data_generators.RandomListGenerator at 0x7f5c21a84940>: 'unknown',
  <pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f5c21a84910>: 'unknown'},
 (100, 75, 20, 5))

In [3]:
bg.gen_mapping

{<pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f5c21a84880>: 'address',
 <pydoxtools.random_data_generators.RandomTextBlockGenerator at 0x7f5c21a84190>: 'unknown',
 <pydoxtools.random_data_generators.RandomListGenerator at 0x7f5c21a84940>: 'unknown',
 <pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f5c21a84910>: 'unknown'}

In [4]:
bg.single(1000, convert_labels=True)

(' 15/02/2016 attend un heureux événement !\nLamy 16:33:45\nStar24\navec mort : il revient ceva le clip de "Moonlight"\n01/10/2018 16:15:09\nattend\nMHD : Le petit prince de l\'Afrotrap va se marier !\n15/12/2017 12:32:10\nStar24\nDécouvrez L trailer de The il of Adaline, le nouveau film de Blake Lively\n20/02/2015 16:13:39\nStar24\nLes Anges 12 : Cloé Cooper dévoile son projet hors télé-réalité\n04/12/2020 17:53:52\n\n L',
 1)

In [5]:
df,y = training.load_labeled_text_block_data(classmap=bg.classmap_inv)
df["label"]=y
df = df.rename(columns={"txt":"text"}).drop(columns="filename")

In [6]:
df.columns

Index(['text', 'label'], dtype='object')

In [7]:
df.label.unique()

array([1, 0])

In [10]:
from datasets import Dataset, Features, Value, ClassLabel
import random

dataset_size=10000

# generate datasets for finetuning
def my_gen():
    seed=random.randint(0,10000000)
    for i in range(1, dataset_size):
        text, label = bg.single(i+seed, convert_labels=True)
        yield {"label": label, "text":text}

class_names = ["address", "unknown"]
features = Features({'text': Value('string'), 'label': ClassLabel(names=class_names)})

dataset = Dataset.from_generator(my_gen, features=features)
dataset.set_format("torch")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /home/tom/.cache/huggingface/datasets/generator/default-63adf7ada54e4f79/0.0.0. Subsequent calls will reuse this data.


In [11]:
val_dataset = Dataset.from_pandas(df,features=features) # this will already convert our labels!
val_dataset.set_format("torch")

In [12]:
dataset, val_dataset

(Dataset({
     features: ['text', 'label'],
     num_rows: 9999
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 2433
 }))

In [13]:
dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['address', 'unknown'], id=None)}

In [14]:
model_name="sentence-transformers/all-MiniLM-L6-v2"
# model_name="bert-base-cased"

In [15]:
%%capture
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(batch):
    tokenized_batch = tokenizer(batch['text'], padding="max_length", truncation=True)
    #print(batch["label"])
    #tokenized_batch["labels"] = [bg.classmap_inv[label] for label in batch["label"]]
    return tokenized_batch

In [16]:
train_dataset = dataset.map(tokenize_function, batched=True)
validation_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2433 [00:00<?, ? examples/s]

In [17]:
# select smaller subdataset
#train_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))
#eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))

In [21]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [36]:
#model = AutoModelForSequenceClassification.from_pretrained(, num_labels=2).to("cuda")
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)#.to("cuda")
mdir="/home/tom/Sync/comcharax_data/models/checkpoint-5800"
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModelForSequenceClassification.from_pretrained(mdir)

loading file vocab.txt from cache at /home/tom/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/7dbbc90392e2f80f3d3c277d6e90027e55de9125/vocab.txt
loading file tokenizer.json from cache at /home/tom/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/7dbbc90392e2f80f3d3c277d6e90027e55de9125/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/tom/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/7dbbc90392e2f80f3d3c277d6e90027e55de9125/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/tom/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/7dbbc90392e2f80f3d3c277d6e90027e55de9125/tokenizer_config.json
loading configuration file /home/tom/Sync/comcharax_data/models/checkpoint-5800/config.json
Model config BertConfig {
  "_name_or_path": "/home/tom/Sync/comcharax_data/mode

In [23]:
from transformers import TrainingArguments

In [24]:
import numpy as np
import evaluate

In [25]:
f1_metric = evaluate.load("f1")
acc = evaluate.load("accuracy")
rec = evaluate.load("recall")
prec = evaluate.load("precision")

In [26]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [27]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        **f1_metric.compute(predictions=predictions, references=labels),
        **acc.compute(predictions=predictions, references=labels),
        **rec.compute(predictions=predictions, references=labels),
        **prec.compute(predictions=predictions, references=labels)
    }

from sklearn.metrics import classification_report
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = classification_report(predictions, labels, output_dict=True)
    label=str(bg.classmap_inv['address'])
    return {
        "address.f1":metrics[label]["f1-score"],
        "accuracy":metrics["accuracy"],
        "address.precision":metrics[label]["precision"],
        "address.recall":metrics[label]["recall"]
    }

In [28]:
#!nvidia-smi

## run actual training

In [29]:
#metric

In [32]:
default_args = {
    "output_dir": "/tmp/models/txtblock",
    "evaluation_strategy": "steps",
    "num_train_epochs": 10,
    "log_level": "info",
    #"label_names": "label",
    "logging_strategy": "steps",
    "logging_steps": 200, 
    "save_strategy":"steps",
    "save_steps": 200,
    "report_to": "none",
    "metric_for_best_model":"address.f1"
}

In [37]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=40,
    #gradient_accumulation_steps=4,
    #gradient_checkpointing=True,
    **default_args)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)
#result = trainer.train()
#print_summary(result)

using `logging_steps` to initialize `eval_steps` to 200
PyTorch: setting up devices


In [34]:
trainer.train()

/home/tom/.cache/pypoetry/virtualenvs/pydoxtools-UuJZOkke-py3.10/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9999
  Num Epochs = 10
  Instantaneous batch size per device = 40
  Total train batch size (w. parallel, distributed & accumulation) = 40
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 22713986
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [38]:
trainer.save_model()

Saving model checkpoint to /tmp/models/txtblock
Configuration saved in /tmp/models/txtblock/config.json
Model weights saved in /tmp/models/txtblock/pytorch_model.bin


In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs